In [ ]:
# -*- coding: utf-8 -*-
"""

    create_topology.py

        Script to process raw node and edge data.

        Workflow:
            - Merge Multilinestrings from power line data       [Complete]
            - Add junction nodes where lines split              [Complete]
            - Add sink nodes to low voltage                     [Complete]
            - Connect supply to substations                     [Complete]
            - Connect high voltage grid to low voltage grid     [Complete]
            - Create bi-directional grid                        [Complete]
            - Save processed spatial data                       [Complete]

"""


#=======================
# Modules
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.wkt import loads
import re
from tqdm import tqdm
tqdm.pandas()

# Add local directory to path
import sys
sys.path.append("../../")

# Import infrasim spatial tools
from JEM.jem.spatial import get_isolated_graphs
from JEM.jem.utils import get_nodal_edges

# Import local copy of snkit
from JEM.snkit.snkit.src.snkit.network import *

# Import local functions
from utils import *
from merge_cost_data import *
from electricity_demand_assignment import *
from merge_elec_consumption_data import *

#=======================
# GLOBAL PARAMS

verbose_flag=True
remove_connected_components = True
connected_component_tolerance = 99

In [ ]:
#=======================
# PROCESSING

# read data
network = read_data()
verbose_print('loaded data',flag=verbose_flag)

# remove known bugs
if 'bug' in network.edges.columns:
    network.edges = network.edges[network.edges.bug != 'true'].reset_index(drop=True)
verbose_print('removed known bugs',flag=verbose_flag)

# merge multilinestrings
network = remove_multiline(network)
verbose_print('removed multilines',flag=verbose_flag)

# delete NoneType
network = remove_nontype(network)
verbose_print('removed NonType',flag=verbose_flag)

# explode multipart linestrings
network = explode_multipart(network)
verbose_print('explode multipart linestrings',flag=verbose_flag)

# save raw data from jps
jps_nodes = network.nodes.copy()
jps_edges = network.edges.copy()

# Merge edges
network = add_endpoints(network)
verbose_print('added end points',flag=verbose_flag)

# add ids
network = add_ids(network)
verbose_print('added IDs',flag=verbose_flag)

# add topology
network = add_topology(network, id_col='id')
verbose_print('added topology',flag=verbose_flag)

# merge using snkit
# network = merge_edges(network,by='asset_type')
verbose_print('merged edges',flag=verbose_flag)

# remove multilines again...
network = remove_multiline(network)

#===
# SNAP LV LINES TO SUBSTATIONS

verbose_print('snapping lines to substations...',flag=verbose_flag)

# LV
lv_voltages = ['24 kV', '12 kV']
# get substations
substations = network.nodes[network.nodes.subtype == 'substation'].geometry
# loop
for s in substations:
    # index edges
    idx_edges = edges_within(s,
                             network.edges[network.edges.voltage.isin(lv_voltages)],
                             distance=40)
    # snap
    for e in idx_edges.itertuples():
        # get current coords of edge
        e_coords = list(e.geometry.coords)
        # get coords of point
        s_coords = list(s.coords)
        # modify first coord of edge to be coord of point (i.e. snap)
        e_coords[0] = s_coords[0]
        # update in edge data
        network.edges.loc[network.edges.index == e.Index, 'geometry'] = LineString(e_coords)

verbose_print('done',flag=verbose_flag)



#===
# ADD JUNCTIONS AND SINKS

verbose_print('adding junctions and sinks...',flag=verbose_flag)

# add endpoints
network = add_endpoints(network)
# update asset_type
network.nodes.loc[~network.nodes.subtype.isin(['sink','junction','sink']),'subtype'] = 'pole'
network.nodes.loc[~network.nodes.asset_type.isin(['sink','junction','sink']),'asset_type'] = 'junction'
# split edges between nodes
network = split_edges_at_nodes(network)
# add ids
network = update_notation(network)
## network.edges.drop(['id','from_id','to_id'],axis=1)
## network = add_id_to_nodes(network)
## network = add_edge_notation(network)
# find true sink nodes
sinks = list(network.edges.to_id.unique())
starts = list(network.edges.from_id.unique())
true_sinks = []
for s in sinks:
    if s in starts:
        continue
    else:
        true_sinks.append(s)

# update true sinks
network.nodes.loc[network.nodes.id.isin(true_sinks),'asset_type'] = 'sink'
network.nodes.loc[network.nodes.id.isin(true_sinks),'subtype'] = 'demand'

# remap asset_type and asset_type from original data
for n in jps_nodes.title:
    network.nodes.loc[network.nodes.title == n, 'asset_type'] = jps_nodes.loc[jps_nodes.title == n].asset_type.iloc[0]
    network.nodes.loc[network.nodes.title == n, 'subtype'] = jps_nodes.loc[jps_nodes.title == n].subtype.iloc[0]

verbose_print('done',flag=verbose_flag)



#===
# CONVERT FALSE JUNCTIONS TO SINKS

verbose_print('converting false junctions...',flag=verbose_flag)

nodes_to_test = network.nodes[network.nodes.subtype.isin(['pole'])].reset_index(drop=True)
for n in nodes_to_test.id:
#for n in ['node_1694']:
    degree = node_connectivity_degree(node=n, network=network)
    if degree == 1:
        # change node asset_type
        network.nodes.loc[network.nodes.id == n, 'asset_type'] = 'sink'
        network.nodes.loc[network.nodes.id == n, 'subtype'] = 'demand'
        # reverse arc direction
        prev_line = network.edges[network.edges.from_id == n].geometry.values[0]
        network.edges.loc[network.edges.from_id == n, 'geometry'] = flip(prev_line)

verbose_print('done',flag=verbose_flag)



#===
# CLEANING/FORMATTING

# add length to line data
network = add_edge_length(network)
verbose_print('added line lengths',flag=verbose_flag)

# remove duplicated
network = remove_duplicates(network)
verbose_print('removed duplicates',flag=verbose_flag)

# change voltage column format
network.edges['voltage_kV'] = network.edges.voltage.str.replace('kV','').astype('int')

# add max/min
network = add_limits_to_edges(network)
verbose_print('added limits to edge flows',flag=verbose_flag)

# double-up edges
network = bidirectional_edges(network)
verbose_print('made edges bidirectional',flag=verbose_flag)

# remove sink-to-sink connections
network = remove_sink_to_sink(network)
verbose_print('removed sink to sinks',flag=verbose_flag)

# add node degree
network = add_nodal_degree(network)
verbose_print('added nodal degrees',flag=verbose_flag)

# drop zero degree sinks
network = remove_stranded_nodes(network)
verbose_print('removed stranded nodes',flag=verbose_flag)

# remove self-loops
network = remove_self_loops(network)
verbose_print('removed self-loops',flag=verbose_flag)

# change asset_type of sinks with >2 degree connectivity
network.nodes.loc[(network.nodes.degree > 2) & \
                  (network.nodes.asset_type == 'sink'), 'asset_type'] = 'junction'

verbose_print('converted sinks of degree>0 to junctions',flag=verbose_flag)



#===
# ADD COST DATA
verbose_print('merging cost data...',flag=verbose_flag)

network = merge_cost_data(network,
                        path_to_costs='../data/costs_and_damages/maximum_damage_values.csv',
                        print_to_console=False)

verbose_print('done',flag=verbose_flag)



#===
# ADD POPULATION
verbose_print('adding population...',flag=verbose_flag)
population = gpd.read_file('../data/population-russell/population.gpkg')
network = assign_pop_to_sinks(network,population)
verbose_print('done',flag=verbose_flag)


#===
# APPEND ELECTRICITY INTENSITIES
network = append_electricity_intensities(network)
verbose_print('appended electricity data',flag=verbose_flag)


#===
# GET CONNECTED COMPONENTS
verbose_print('getting connected components...',flag=verbose_flag)

network = add_component_ids(network)
# remove
if not remove_connected_components:
    pass
else:
    graphs_to_remove = network.edges.loc[network.edges.component_id > connected_component_tolerance]
    nodes_to_remove = graphs_to_remove.from_id.to_list() + graphs_to_remove.to_id.to_list()
    edges_to_remove = graphs_to_remove.id.to_list()
    # drop
    network.nodes = network.nodes.loc[~network.nodes.id.isin(nodes_to_remove)].reset_index(drop=True)
    network.edges = network.edges.loc[~network.edges.id.isin(edges_to_remove)].reset_index(drop=True)
# Update network notation
network = update_notation(network)

verbose_print('done',flag=verbose_flag)


#===
# ADD CAPACITY ATTRIBUTES
verbose_print('adding capacity attributes to nodes...',flag=verbose_flag)

def nodal_capacity_from_edges(node,network):
    nodal_edges = get_nodal_edges(network,node).id.to_list()
    return network.edges.loc[network.edges.id.isin(nodal_edges)]['max'].max()


network.nodes['capacity'] \
    = network.nodes.progress_apply(
        lambda x: nodal_capacity_from_edges(x['id'],network) \
            if pd.isnull(x['capacity']) else x['capacity'], axis=1 )

verbose_print('done',flag=verbose_flag)


#===
# ADD TOTAL COSTS

# edges
network.edges['cost_min'] = network.edges['uc_min'] * network.edges['max'] * network.edges['length']
network.edges['cost_max'] = network.edges['uc_max'] * network.edges['max'] * network.edges['length']
network.edges['cost_avg'] = network.edges['uc_avg'] * network.edges['max'] * network.edges['length']
network.edges['cost_uom'] = '$US'

# nodes
network.nodes['cost_min'] = network.nodes['uc_min'] * network.nodes['capacity']
network.nodes['cost_max'] = network.nodes['uc_max'] * network.nodes['capacity']
network.nodes['cost_avg'] = network.nodes['uc_avg'] * network.nodes['capacity']
network.nodes['cost_uom'] = '$US'


#===
# REINDEX
network.edges = network.edges[['id', 'asset_type', 'from_id', 'to_id', 'from_type', 'to_type',
                               'voltage_kV', 'losses', 'length', 'min', 'max', 
                               'uc_min','uc_max', 'uc_avg','uc_uom',
                               'cost_min','cost_max', 'cost_avg','cost_uom',
                               'name', 'parish','source', 'component_id', 'geometry']]

network.nodes = network.nodes[['id','asset_type','subtype','capacity','population',
                              'uc_min','uc_max','uc_avg','uc_uom',
                              'cost_min','cost_max', 'cost_avg','cost_uom',
                              'ei', 'ei_uom','degree','parish','title','source','geometry']]

verbose_print('re-indexed data',flag=verbose_flag)

#===
# SAVE DATA
verbose_print('saving...',flag=verbose_flag)

save_data(network)

verbose_print('create_toplogy finished',flag=verbose_flag)

In [ ]:
#===
# ADD POPULATION
verbose_print('adding population...',flag=verbose_flag)
population = gpd.read_file('../data/population-russell/population.gpkg')
network = assign_pop_to_sinks(network,population)
verbose_print('done',flag=verbose_flag)

In [ ]:
# ADD POPULATION
population_dataframe = gpd.read_file('../data/population-russell/population.gpkg')
# get sinks
sinks = network.nodes[network.nodes['asset_type'] == 'sink']
# change crs
epsg=3448,
node_id_column='id'
population_value_column='population'
nodes_dataframe = sinks.copy()
#sinks = sinks.to_crs(epsg=epsg)
#pop_bound = pop_bound.to_crs(epsg=epsg)

#------
# assign_node_weights_by_population_proximity

# load provinces and get geometry of the right population_dataframe
sindex_population_dataframe = population_dataframe.sindex

# create Voronoi polygons for the nodes
xy_list = []
for iter_, values in nodes_dataframe.iterrows():
    xy = list(values.geometry.coords)
    xy_list += [list(xy[0])]

vor = Voronoi(np.array(xy_list))
regions, vertices = voronoi_finite_polygons_2d(vor)
min_x = vor.min_bound[0] - 0.1
max_x = vor.max_bound[0] + 0.1
min_y = vor.min_bound[1] - 0.1
max_y = vor.max_bound[1] + 0.1

mins = np.tile((min_x, min_y), (vertices.shape[0], 1))
bounded_vertices = np.max((vertices, mins), axis=0)
maxs = np.tile((max_x, max_y), (vertices.shape[0], 1))
bounded_vertices = np.min((bounded_vertices, maxs), axis=0)

box = Polygon([[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]])

poly_list = []
for region in regions:
    polygon = vertices[region]
    # Clipping polygon
    poly = Polygon(polygon)
    poly = poly.intersection(box)
    poly_list.append(poly)

poly_index = list(np.arange(0, len(poly_list), 1))
poly_df = pd.DataFrame(list(zip(poly_index, poly_list)),
                               columns=['gid', 'geometry'])

gdf_voronoi = gpd.GeoDataFrame(poly_df, geometry = 'geometry') #,crs=f'epsg:{epsg}'

gdf_voronoi['areas'] = gdf_voronoi.geometry.area #gdf_voronoi.progress_apply(lambda x:x.geometry.area,axis=1)

gdf_voronoi = gdf_voronoi.head(100)

def extract_nodes_within_gdf(x,nodes_dataframe,node_id_column):
    a = nodes_dataframe.loc[ list(nodes_dataframe.geometry.within(x)) ]
    if len(a.index) > 0:
        v = a[node_id_column].values[0]
    else:
        v = np.nan
    return v

gdf_voronoi[node_id_column] = gdf_voronoi.progress_apply(
        lambda row: extract_nodes_within_gdf(row['geometry'],nodes_dataframe,node_id_column),axis=1)

gdf_voronoi[population_value_column] = 0
gdf_voronoi = assign_value_in_area_proportions(population_dataframe, gdf_voronoi, population_value_column)
gdf_voronoi = gdf_voronoi[~(gdf_voronoi[node_id_column] == '')]

gdf_pops = gdf_voronoi#[[node_id_column, population_value_column]]
del gdf_voronoi, poly_list, poly_df

nodes_dataframe = pd.merge(nodes_dataframe, gdf_pops, how='left', on=[node_id_column])
del gdf_pops, population_dataframe

new_nodes = nodes_dataframe

pop_mapped = new_nodes[['id','population']].set_index('id')['population'].to_dict()
# reassign
network.nodes['population'] = network.nodes['id'].map(pop_mapped).fillna(0)

In [ ]:
node_id_column

In [ ]:
network.nodes

In [ ]:
import sys
import os
import json
import pandas as pd
import geopandas as gpd
from scipy.spatial import Voronoi
from shapely.geometry import Polygon, shape
# workaround for geopandas >0.9 until snkit #37 and geopandas #1977 are fixed
gpd._compat.USE_PYGEOS = False
import fiona
import numpy as np
import snkit
#import snkit_network
from tqdm import tqdm
#tqdm.pandas()

def assign_pop_to_sinks(network,pop_bound,
                        epsg=3448,nodal_id='id',pop_id='population'):
    '''Assign population to sink nodes
    '''
    # get sinks
    sinks = network.nodes[network.nodes['asset_type'] == 'sink']
    # change crs
    sinks = sinks.to_crs(epsg=epsg)
    pop_bound = pop_bound.to_crs(epsg=3448)
    # compute
    new_nodes = assign_node_weights_by_population_proximity(sinks,
                                                            pop_bound,
                                                            nodal_id,
                                                            pop_id,
                                                            epsg=epsg,
                                                            save=True,
                                                            voronoi_path='../data/spatial/electricity_voronoi.shp',
                                                            )
    #remap
    pop_mapped = new_nodes[['id','population']].set_index('id')['population'].to_dict()
    # reassign
    network.nodes['population'] = network.nodes['id'].map(pop_mapped).fillna(0)
    return network


def assign_node_weights_by_population_proximity(nodes_dataframe,
                        population_dataframe,
                        node_id_column,population_value_column,epsg=4326,**kwargs):
    """Assign weights to nodes based on their nearest populations

        - By finding the population that intersect with the Voronoi extents of nodes

    Parameters
        - nodes_dataframe - Geodataframe of the nodes
        - population_dataframe - Geodataframe of the population
        - nodes_id_column - String name of node ID column
        - population_value_column - String name of column containing population values

    Outputs
        - nodes - Geopandas dataframe of nodes with new column called population
    """

    # load provinces and get geometry of the right population_dataframe
    sindex_population_dataframe = population_dataframe.sindex

    # create Voronoi polygons for the nodes
    xy_list = []
    for iter_, values in nodes_dataframe.iterrows():
        xy = list(values.geometry.coords)
        xy_list += [list(xy[0])]

    vor = Voronoi(np.array(xy_list))
    regions, vertices = voronoi_finite_polygons_2d(vor)
    min_x = vor.min_bound[0] - 0.1
    max_x = vor.max_bound[0] + 0.1
    min_y = vor.min_bound[1] - 0.1
    max_y = vor.max_bound[1] + 0.1

    mins = np.tile((min_x, min_y), (vertices.shape[0], 1))
    bounded_vertices = np.max((vertices, mins), axis=0)
    maxs = np.tile((max_x, max_y), (vertices.shape[0], 1))
    bounded_vertices = np.min((bounded_vertices, maxs), axis=0)

    box = Polygon([[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]])

    poly_list = []
    for region in regions:
        polygon = vertices[region]
        # Clipping polygon
        poly = Polygon(polygon)
        poly = poly.intersection(box)
        poly_list.append(poly)

    poly_index = list(np.arange(0, len(poly_list), 1))
    poly_df = pd.DataFrame(list(zip(poly_index, poly_list)),
                                   columns=['gid', 'geometry'])
    
    gdf_voronoi = gpd.GeoDataFrame(poly_df, geometry = 'geometry',crs=f'epsg:{epsg}')    
    gdf_voronoi['areas'] = gdf_voronoi.progress_apply(lambda x:x.geometry.area,axis=1)
    gdf_voronoi[node_id_column] = gdf_voronoi.progress_apply(
        lambda x: extract_nodes_within_gdf(x, nodes_dataframe, node_id_column), axis=1)
    if not kwargs.get('save',False):
        pass
    else:
        gdf_voronoi.to_file(kwargs.get('voronoi_path','voronoi-output.shp'))

    gdf_voronoi[population_value_column] = 0
    gdf_voronoi = assign_value_in_area_proportions(population_dataframe, gdf_voronoi, population_value_column)
    gdf_voronoi = gdf_voronoi[~(gdf_voronoi[node_id_column] == '')]

    gdf_pops = gdf_voronoi#[[node_id_column, population_value_column]]
    del gdf_voronoi, poly_list, poly_df

    nodes_dataframe = pd.merge(nodes_dataframe, gdf_pops, how='left', on=[node_id_column])
    del gdf_pops, population_dataframe

    return nodes_dataframe


def voronoi_finite_polygons_2d(vor, radius=None):
    """Reconstruct infinite voronoi regions in a 2D diagram to finite regions.

    Source: https://stackoverflow.com/questions/36063533/clipping-a-voronoi-diagram-python

    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'

    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1]  # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

def assign_value_in_area_proportions(poly_1_gpd, poly_2_gpd, poly_attribute):
    poly_1_sindex = poly_1_gpd.sindex
    for p_2_index, polys_2 in poly_2_gpd.iterrows():
        poly2_attr = 0
        intersected_polys = poly_1_gpd.iloc[list(
            poly_1_sindex.intersection(polys_2.geometry.bounds))]
        for p_1_index, polys_1 in intersected_polys.iterrows():
            if (polys_2['geometry'].intersects(polys_1['geometry']) is True) and (polys_1.geometry.is_valid is True) and (polys_2.geometry.is_valid is True):
                poly2_attr += polys_1[poly_attribute]*polys_2['geometry'].intersection(
                    polys_1['geometry']).area/polys_1['geometry'].area

        poly_2_gpd.loc[p_2_index, poly_attribute] = poly2_attr

    return poly_2_gpd

In [ ]:
def extract_nodes_within_gdf2(input_df,input_nodes,column_name):
    for x in tqdm(input_df.geometry):
        a = input_nodes.loc[list(input_nodes.geometry.within(x))]
        if len(a.index) > 0:
            v = a[column_name].values[0]
        else:
            v = np.nan
        input_df.loc[input_df.geometry == x,column_name] = v
    return input_df


gdf_voronoi = extract_nodes_within_gdf2(gdf_voronoi,nodes_dataframe,node_id_column)

In [ ]:
def extract_nodes_within_gdf(x,nodes_dataframe,node_id_column):
    a = nodes_dataframe.loc[ list(nodes_dataframe.geometry.within(x)) ]
    if len(a.index) > 0:
        v = a[column_name].values[0]
    else:
        v = np.nan
    return v

gdf_voronoi['node_id'] = 
    gdf_voronoi.progress_apply(
        lambda row: extract_nodes_within_gdf(row['geometry'],nodes_dataframe,node_id_column),axis=1)
        
        
        
